# Fine-tuning with Dreambooth

Made to use on colab's GPU

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%ls

drive/  sample_data/


In [3]:
!git clone https://github.com/huggingface/diffusers

Cloning into 'diffusers'...
remote: Enumerating objects: 56940, done.
remote: Counting objects: 100% (1120/1120), done.
remote: Compressing objects: 100% (581/581), done.
remote: Total 56940 (delta 691), reused 749 (delta 433), pack-reused 55820
Receiving objects: 100% (56940/56940), 39.29 MiB | 14.48 MiB/s, done.
Resolving deltas: 100% (40936/40936), done.


In [4]:
%cd diffusers
!pip install .
%cd examples/dreambooth
!pip install -r requirements.txt
!accelerate config default

/content/diffusers
Processing /content/diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.27.0.dev0-py3-none-any.whl size=1982931 sha256=d62418d21b1887ac8c48a30bd38e096c3f0a08dbcefbbffb566f10f030492c98
  Stored in directory: /tmp/pip-ephem-wheel-cache-o0anm0on/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers
/content/diffusers/examples/dreambooth
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 24.3 MB/s eta 0:00:00
accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
# Links to go from dreambooth to drive from here to save the results
db_to_drive_path = './../../../drive/MyDrive/SketchToReality/'
drive_to_db_path = './../../../../diffusers/examples/dreambooth' # In case, to come back :)

In [5]:
%ls

README.md              requirements.txt             train_dreambooth_flax.py
README_sdxl.md         test_dreambooth_lora_edm.py  train_dreambooth_lora.py
requirements_flax.txt  test_dreambooth_lora.py      train_dreambooth_lora_sdxl.py
requirements_sdxl.txt  test_dreambooth.py           train_dreambooth.py


__________________________
**VERY IMPORTANT**

1. Colab doesn't support the standard commands on regular terminals.
To avoid errors when running dreambooth with the command line in the huggingface tutorial, change `train_dreambooth.py`at the 3rd line before the end:

- Delete the `if __name__ == 'main':`
- Unindent the lines below
At the end of the file, you should have
```
args=parseargs()
main(args)
```
2. Don't forget to go on the 8Gb GPU version on the huggingface tutorial, as 16Gb won't be supported on colab.

3. Make sure your data and results paths are valid and that the folders do not already contain data

______________________________

In [8]:
!pip install -q diffusers transformers accelerate peft # Trick found online to make it work
!pip install xformers
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [12]:
# Show possible parameters to pass
!accelerate launch train_dreambooth.py --help

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-03-08 22:18:47.540100: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 22:18:47.540182: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 22:18:47.542260: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:

In [17]:
!accelerate launch train_dreambooth.py \
 --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
 --instance_data_dir="./../../../drive/MyDrive/SketchToReality/dreambooth_cat" \
 --instance_prompt="a photo of a sks cat" \
 --output_dir "./../../../drive/MyDrive/SketchToReality/dreambooth_cat_results" \
 --use_8bit_adam \
 --gradient_checkpointing \
 --enable_xformers_memory_efficient_attention \
 --set_grads_to_none \
 --train_batch_size=2 \
 --gradient_accumulation_steps=1 \
 --learning_rate=5e-6 \
 --lr_scheduler="constant" \
 --lr_warmup_steps=0 \
 --max_train_steps=400

# These parameters bring an error - I don't know why
  # --num_train_epochs= 5 \
  # --checkpointing_steps= 3 \

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-03-08 22:28:31.848059: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-08 22:28:31.848125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-08 22:28:31.849998: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc: